# Scrape Data science books available on amazon using selenium and python 

In [1]:
#Necessary library importation
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC

service = Service(executable_path='path\chromedriver.exe')
driver = webdriver.Chrome(executable_path='path\chromedriver.exe')
driver.implicitly_wait(1)

link = ""
product_name = []
product_price = []
product_ratings = []
product_ratings_num = []
product_link = []
# for loop to iterate over each pages                         
for i in range(1,51):
    driver.get(f'https://www.amazon.com/s?k=Data+science+books&i=stripbooks&rh=n%3A283155%2Cp_n_feature_nine_browse-bin%3A3291437011&dc&page={i}&crid=3RJJC3J5VZT3C&qid=1662414684&rnid=3291435011&sprefix=data+science+books%2Caps%2C219&ref=sr_pg_1')
    items = wait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//div[contains(@class, "s-result-item s-asin")]')))
    for item in items:
        # Get the name of book
        name = item.find_element(By.XPATH, './/span[@class="a-size-base-plus a-color-base a-text-normal"]')
        product_name.append(name.text)

        # Get the price
        whole_price = item.find_elements(By.XPATH, './/span[@class="a-price-whole"]')
        fraction_price = item.find_elements(By.XPATH, './/span[@class="a-price-fraction"]')

        if whole_price != [] and fraction_price != []:
            price = '.'.join([whole_price[0].text, fraction_price[0].text])
        else:
            price = 0
        product_price.append(price)

        # find ratings box
        ratings_box = item.find_elements(By.XPATH, './/div[@class="a-row a-size-small"]/span')

        # find ratings and ratings_num
        if ratings_box != []:
            ratings = ratings_box[0].get_attribute('aria-label')
            ratings_num = ratings_box[1].get_attribute('aria-label')
        else:
            ratings, ratings_num = 0, 0

        product_ratings.append(ratings)
        product_ratings_num.append(str(ratings_num))

        # Get the link
        link = item.find_element(By.XPATH, './/a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]').get_attribute("href")
        product_link.append(link)

driver.quit()

# Print the information to check if the data is collected correctly
#print(product_name)
#print(product_price)
#print(product_ratings)
#print(product_ratings_num)
#print(product_link)

In [2]:
# Store the data into a dataframe and save them in a csv file
dict_df = {"Nme of book":product_name, "Price":product_price, "Rating stars":product_ratings, "Rating count": product_ratings_num, "Link to the book":product_link }
df = pd.DataFrame(dict_df)
df.to_csv("Data/Data_science_book.csv")